In [1]:
import os
import h5py
import shutil
import numpy as np

from pathlib import Path
from utils import phi_shift_and_flipping, std_phi

import utils_HDF5 as utils_h5

MAX_CONSTI = {
    'Jet': 50,
    'Tower': 250,
    'Track': 150,
    'Photon': 2,
    'Lepton': 4,
}

In [2]:
def pt_scale(pt):
    return np.sqrt(0.052*pt*pt + 1.502*pt)

# Merge GGF & VBF samples

In [3]:
files = [f'./data/GGF-{i:02}.h5' for i in range(3, 5)]

merged_file = utils_h5.merge_h5_file(*files)
new_name = './data/GGF.h5'

os.rename(merged_file, new_name)

files = [f'./data/VBF-{i:02}.h5' for i in range(3, 4)]

merged_file = utils_h5.merge_h5_file(*files)
new_name = './data/VBF.h5'

os.rename(merged_file, new_name)

'./data/GGF-03.h5' and ('./data/GGF-04.h5',) are same structure, can be merged.
./data/GGF-03_merged.h5 not exist. Copy ./data/GGF-03.h5 to ./data/GGF-03_merged.h5
Size of ./data/GGF-03.h5: 81263
Size of ./data/GGF-04.h5: 81066
Size of ./data/GGF-03_merged.h5: 162329
'./data/VBF-03.h5' and () are same structure, can be merged.
./data/VBF-03_merged.h5 not exist. Copy ./data/VBF-03.h5 to ./data/VBF-03_merged.h5
Size of ./data/VBF-03.h5: 332187


In [3]:
files = [f'./data/ZZ_4l/GGF_ZZ-{i:02}.h5' for i in range(3, 6)]

merged_file = utils_h5.merge_h5_file(*files)
new_name = './data/ZZ_4l/GGF.h5'

os.rename(merged_file, new_name)

files = [f'./data/ZZ_4l/VBF_ZZ-{i:02}.h5' for i in range(3, 5)]

merged_file = utils_h5.merge_h5_file(*files)
new_name = './data/ZZ_4l/VBF.h5'

os.rename(merged_file, new_name)

'./data/ZZ_4l/GGF_ZZ-03.h5' and ('./data/ZZ_4l/GGF_ZZ-04.h5', './data/ZZ_4l/GGF_ZZ-05.h5') are same structure, can be merged.
./data/ZZ_4l/GGF_ZZ-03_merged.h5 not exist. Copy ./data/ZZ_4l/GGF_ZZ-03.h5 to ./data/ZZ_4l/GGF_ZZ-03_merged.h5
Size of ./data/ZZ_4l/GGF_ZZ-03.h5: 65399
Size of ./data/ZZ_4l/GGF_ZZ-04.h5: 65407
Size of ./data/ZZ_4l/GGF_ZZ-03_merged.h5: 130806
Size of ./data/ZZ_4l/GGF_ZZ-05.h5: 65401
Size of ./data/ZZ_4l/GGF_ZZ-03_merged.h5: 196207
'./data/ZZ_4l/VBF_ZZ-03.h5' and ('./data/ZZ_4l/VBF_ZZ-04.h5',) are same structure, can be merged.
./data/ZZ_4l/VBF_ZZ-03_merged.h5 not exist. Copy ./data/ZZ_4l/VBF_ZZ-03.h5 to ./data/ZZ_4l/VBF_ZZ-03_merged.h5
Size of ./data/ZZ_4l/VBF_ZZ-03.h5: 133162
Size of ./data/ZZ_4l/VBF_ZZ-04.h5: 132982
Size of ./data/ZZ_4l/VBF_ZZ-03_merged.h5: 266144


# Split SR (VBF) and BR (GGF) samples 

In [3]:
def get_dataset_keys(f):
    keys = []
    f.visit(lambda key : keys.append(key) if isinstance(f[key], h5py.Dataset) else None)
    return keys

def create_dataset(f, nevent, MAX_CONSTI):

    f.create_dataset('J1/mask', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='|b1')
    f.create_dataset('J1/pt', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J1/eta', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J1/phi', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J1/flavor', (nevent,), maxshape=(None,), dtype='<i8')

    f.create_dataset('J2/mask', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='|b1')
    f.create_dataset('J2/pt', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J2/eta', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J2/phi', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J2/flavor', (nevent,), maxshape=(None,), dtype='<i8')

    f.create_dataset('TOWER/mask', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='|b1')
    f.create_dataset('TOWER/pt', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='<f4')
    f.create_dataset('TOWER/eta', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='<f4')
    f.create_dataset('TOWER/phi', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='<f4')

    f.create_dataset('TRACK/mask', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='|b1')
    f.create_dataset('TRACK/pt', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='<f4')
    f.create_dataset('TRACK/eta', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='<f4')
    f.create_dataset('TRACK/phi', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='<f4')

    f.create_dataset('PHOTON/pt', (nevent, MAX_CONSTI['Photon']), maxshape=(None, MAX_CONSTI['Photon']), dtype='<f4')
    f.create_dataset('PHOTON/eta', (nevent, MAX_CONSTI['Photon']), maxshape=(None, MAX_CONSTI['Photon']), dtype='<f4')
    f.create_dataset('PHOTON/phi', (nevent, MAX_CONSTI['Photon']), maxshape=(None, MAX_CONSTI['Photon']), dtype='<f4')

    f.create_dataset('EVENT/mjj', (nevent,), maxshape=(None,), dtype='<f4')
    f.create_dataset('EVENT/deta', (nevent,), maxshape=(None,), dtype='<f4')
    f.create_dataset('EVENT/type', (nevent,), maxshape=(None,), dtype='<i8')

def create_dataset_ZZ(f, nevent, MAX_CONSTI):

    f.create_dataset('J1/mask', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='|b1')
    f.create_dataset('J1/pt', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J1/eta', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J1/phi', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J1/flavor', (nevent,), maxshape=(None,), dtype='<i8')

    f.create_dataset('J2/mask', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='|b1')
    f.create_dataset('J2/pt', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J2/eta', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J2/phi', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J2/flavor', (nevent,), maxshape=(None,), dtype='<i8')

    f.create_dataset('TOWER/mask', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='|b1')
    f.create_dataset('TOWER/pt', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='<f4')
    f.create_dataset('TOWER/eta', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='<f4')
    f.create_dataset('TOWER/phi', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='<f4')

    f.create_dataset('TRACK/mask', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='|b1')
    f.create_dataset('TRACK/pt', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='<f4')
    f.create_dataset('TRACK/eta', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='<f4')
    f.create_dataset('TRACK/phi', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='<f4')

    f.create_dataset('LEPTON/pt', (nevent, MAX_CONSTI['Lepton']), maxshape=(None, MAX_CONSTI['Lepton']), dtype='<f4')
    f.create_dataset('LEPTON/eta', (nevent, MAX_CONSTI['Lepton']), maxshape=(None, MAX_CONSTI['Lepton']), dtype='<f4')
    f.create_dataset('LEPTON/phi', (nevent, MAX_CONSTI['Lepton']), maxshape=(None, MAX_CONSTI['Lepton']), dtype='<f4')
    f.create_dataset('LEPTON/flavor', (nevent, MAX_CONSTI['Lepton']), maxshape=(None, MAX_CONSTI['Lepton']), dtype='<i8')

    f.create_dataset('EVENT/mjj', (nevent,), maxshape=(None,), dtype='<f4')
    f.create_dataset('EVENT/deta', (nevent,), maxshape=(None,), dtype='<f4')
    f.create_dataset('EVENT/type', (nevent,), maxshape=(None,), dtype='<i8')

In [4]:
def split_SR_BR(h5_path, output_path, cut_type='mjj', cut_value=300, mode='di-photon'):
    # read data
    with h5py.File(h5_path, 'r') as f:

        mjj = f[f'EVENT/mjj'][:]
        deta = f[f'EVENT/deta'][:]
        j1_flavor = f[f'J1/flavor'][:]
        j2_flavor = f[f'J2/flavor'][:]
        if cut_type == 'mjj':
            SR_range = mjj > cut_value
            BR_range = mjj < cut_value
        elif cut_type == 'deta':
            SR_range = deta > cut_value
            BR_range = deta < cut_value
        elif cut_type == 'mjj_deta':
            SR_range = (mjj > cut_value[0]) & (deta > cut_value[1])
            BR_range = (mjj < cut_value[0]) | (deta < cut_value[1])
        elif cut_type == 'gluon_jet':
            gluon_jet = (j1_flavor == 21).astype(int) + (j2_flavor == 21).astype(int)
            SR_range = gluon_jet >= cut_value
            BR_range = gluon_jet < cut_value
        elif cut_type == 'quark_jet':
            quark_jet = (j1_flavor < 6).astype(int) + (j2_flavor < 6).astype(int)
            SR_range = quark_jet >= cut_value
            BR_range = quark_jet < cut_value
        elif cut_type == 'quark_gluon_jet':
            gluon_jet = (j1_flavor == 21).astype(int) + (j2_flavor == 21).astype(int)
            quark_jet = (j1_flavor < 6).astype(int) + (j2_flavor < 6).astype(int)
            SR_range = quark_jet == cut_value
            BR_range = gluon_jet == cut_value
        else:
            raise ValueError(f'cut_type {cut_type} not supported')

        
        root, _ = os.path.splitext(output_path)
        output_path = Path(output_path)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        root = output_path.with_suffix('')
        SR_path = f'{root}_in_SR.h5'
        BR_path = f'{root}_in_BR.h5'

        with h5py.File(SR_path, 'w') as f_SR, h5py.File(BR_path, 'w') as f_SB:

            if mode == 'di-photon':
                create_dataset(f_SR, SR_range.sum(), MAX_CONSTI)
                create_dataset(f_SB, BR_range.sum(), MAX_CONSTI)
            elif mode == 'ZZ_4l':
                create_dataset_ZZ(f_SR, SR_range.sum(), MAX_CONSTI)
                create_dataset_ZZ(f_SB, BR_range.sum(), MAX_CONSTI)
            else:
                raise ValueError(f'mode {mode} not supported')

            keys = get_dataset_keys(f_SR)
            for key in keys:
                f_SR[key][:] = f[key][:][SR_range]
                f_SB[key][:] = f[key][:][BR_range]

In [5]:
cut_type = 'mjj'
cut_value = 225

h5_path = 'data/GGF.h5'
output_path = 'data/mjj_225_cut/GGF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

h5_path = 'data/VBF.h5'
output_path = 'data/mjj_225_cut/VBF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

In [6]:
cut_type = 'deta'
cut_value = 2.3

h5_path = 'data/GGF.h5'
output_path = 'data/deta_2.3_cut/GGF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

h5_path = 'data/VBF.h5'
output_path = 'data/deta_2.3_cut/VBF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

In [7]:
cut_type = 'mjj_deta'
cut_value = (225, 2.3)

h5_path = 'data/GGF.h5'
output_path = 'data/mjj_225_deta_2.3_cut/GGF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

h5_path = 'data/VBF.h5'
output_path = 'data/mjj_225_deta_2.3_cut/VBF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

In [11]:
cut_type = 'gluon_jet'
cut_value = 2

h5_path = 'data/GGF.h5'
output_path = f'data/{cut_type}_{cut_value}_cut/GGF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

h5_path = 'data/VBF.h5'
output_path = f'data/{cut_type}_{cut_value}_cut/VBF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

cut_value = 1

h5_path = 'data/GGF.h5'
output_path = f'data/{cut_type}_{cut_value}_cut/GGF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

h5_path = 'data/VBF.h5'
output_path = f'data/{cut_type}_{cut_value}_cut/VBF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

In [5]:
cut_type = 'quark_jet'
cut_value = 2

h5_path = 'data/GGF.h5'
output_path = f'data/{cut_type}_{cut_value}_cut/GGF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

h5_path = 'data/VBF.h5'
output_path = f'data/{cut_type}_{cut_value}_cut/VBF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

cut_value = 1

h5_path = 'data/GGF.h5'
output_path = f'data/{cut_type}_{cut_value}_cut/GGF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

h5_path = 'data/VBF.h5'
output_path = f'data/{cut_type}_{cut_value}_cut/VBF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

In [5]:
cut_type = 'quark_gluon_jet'
cut_value = 2

h5_path = 'data/GGF.h5'
output_path = f'data/{cut_type}_{cut_value}_cut/GGF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

h5_path = 'data/VBF.h5'
output_path = f'data/{cut_type}_{cut_value}_cut/VBF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value)

In [5]:
cut_type = 'quark_jet'
cut_value = 2

h5_path = 'data/ZZ_4l/GGF.h5'
output_path = f'data/ZZ_4l/{cut_type}_{cut_value}_cut/GGF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value, mode='ZZ_4l')

h5_path = 'data/ZZ_4l/VBF.h5'
output_path = f'data/ZZ_4l/{cut_type}_{cut_value}_cut/VBF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value, mode='ZZ_4l')

cut_value = 1

h5_path = 'data/ZZ_4l/GGF.h5'
output_path = f'data/ZZ_4l/{cut_type}_{cut_value}_cut/GGF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value, mode='ZZ_4l')

h5_path = 'data/ZZ_4l/VBF.h5'
output_path = f'data/ZZ_4l/{cut_type}_{cut_value}_cut/VBF.h5'
split_SR_BR(h5_path, output_path, cut_type, cut_value, mode='ZZ_4l')

# $\phi$ shifting and flipping

In [4]:
def to_event_image_h5(h5_path, out_h5):
    
    out_h5 = Path(out_h5)
    out_h5.parent.mkdir(parents=True, exist_ok=True)

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)

        event_pt = np.concatenate([f_out['TOWER/pt'][:], f_out['TRACK/pt'][:], f_out['PHOTON/pt'][:]], axis=1)
        event_eta = np.concatenate([f_out['TOWER/eta'][:], f_out['TRACK/eta'][:], f_out['PHOTON/eta'][:]], axis=1)
        event_phi = np.concatenate([f_out['TOWER/phi'][:], f_out['TRACK/phi'][:], f_out['PHOTON/phi'][:]], axis=1)

        _, new_eta, new_phi = phi_shift_and_flipping(event_pt, event_eta, event_phi)

        f_out['TOWER/eta'][:] = new_eta[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/eta'][:] = new_eta[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['PHOTON/eta'][:] = new_eta[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

        f_out['TOWER/phi'][:] = new_phi[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/phi'][:] = new_phi[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['PHOTON/phi'][:] = new_phi[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

def to_event_image_h5_ZZ(h5_path, out_h5):
    
    out_h5 = Path(out_h5)
    out_h5.parent.mkdir(parents=True, exist_ok=True)

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)

        event_pt = np.concatenate([f_out['TOWER/pt'][:], f_out['TRACK/pt'][:], f_out['LEPTON/pt'][:]], axis=1)
        event_eta = np.concatenate([f_out['TOWER/eta'][:], f_out['TRACK/eta'][:], f_out['LEPTON/eta'][:]], axis=1)
        event_phi = np.concatenate([f_out['TOWER/phi'][:], f_out['TRACK/phi'][:], f_out['LEPTON/phi'][:]], axis=1)

        _, new_eta, new_phi = phi_shift_and_flipping(event_pt, event_eta, event_phi)

        f_out['TOWER/eta'][:] = new_eta[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/eta'][:] = new_eta[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['LEPTON/eta'][:] = new_eta[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

        f_out['TOWER/phi'][:] = new_phi[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/phi'][:] = new_phi[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['LEPTON/phi'][:] = new_phi[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

In [9]:
for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/mjj_cut/{name}.h5'
    out_h5 = f'./data/mjj_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/deta_cut/{name}.h5'
    out_h5 = f'./data/deta_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/mjj_deta_cut/{name}.h5'
    out_h5 = f'./data/mjj_deta_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

data/mjj_cut/pre-processing/VBF_in_SR.h5
data/mjj_cut/pre-processing/VBF_in_BR.h5
data/mjj_cut/pre-processing/GGF_in_SR.h5
data/mjj_cut/pre-processing/GGF_in_BR.h5
data/deta_cut/pre-processing/VBF_in_SR.h5
data/deta_cut/pre-processing/VBF_in_BR.h5
data/deta_cut/pre-processing/GGF_in_SR.h5
data/deta_cut/pre-processing/GGF_in_BR.h5
data/mjj_deta_cut/pre-processing/VBF_in_SR.h5
data/mjj_deta_cut/pre-processing/VBF_in_BR.h5
data/mjj_deta_cut/pre-processing/GGF_in_SR.h5
data/mjj_deta_cut/pre-processing/GGF_in_BR.h5


In [10]:
for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/mjj_225_cut/{name}.h5'
    out_h5 = f'./data/mjj_225_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/deta_2.3_cut/{name}.h5'
    out_h5 = f'./data/deta_2.3_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/mjj_225_deta_2.3_cut/{name}.h5'
    out_h5 = f'./data/mjj_225_deta_2.3_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

data/mjj_225_cut/pre-processing/VBF_in_SR.h5
data/mjj_225_cut/pre-processing/VBF_in_BR.h5
data/mjj_225_cut/pre-processing/GGF_in_SR.h5
data/mjj_225_cut/pre-processing/GGF_in_BR.h5
data/deta_2.3_cut/pre-processing/VBF_in_SR.h5
data/deta_2.3_cut/pre-processing/VBF_in_BR.h5
data/deta_2.3_cut/pre-processing/GGF_in_SR.h5
data/deta_2.3_cut/pre-processing/GGF_in_BR.h5
data/mjj_225_deta_2.3_cut/pre-processing/VBF_in_SR.h5
data/mjj_225_deta_2.3_cut/pre-processing/VBF_in_BR.h5
data/mjj_225_deta_2.3_cut/pre-processing/GGF_in_SR.h5
data/mjj_225_deta_2.3_cut/pre-processing/GGF_in_BR.h5


In [3]:
for name in ['VBF', 'GGF']:
    h5_path = f'./data/{name}.h5'
    out_h5 = f'./data/supervised/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

data/supervised/pre-processing/VBF.h5
data/supervised/pre-processing/GGF.h5


In [13]:
for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/gluon_jet_2_cut/{name}.h5'
    out_h5 = f'./data/gluon_jet_2_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/gluon_jet_1_cut/{name}.h5'
    out_h5 = f'./data/gluon_jet_1_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

data/gluon_jet_2_cut/pre-processing/VBF_in_SR.h5
data/gluon_jet_2_cut/pre-processing/VBF_in_BR.h5
data/gluon_jet_2_cut/pre-processing/GGF_in_SR.h5
data/gluon_jet_2_cut/pre-processing/GGF_in_BR.h5
data/gluon_jet_1_cut/pre-processing/VBF_in_SR.h5
data/gluon_jet_1_cut/pre-processing/VBF_in_BR.h5
data/gluon_jet_1_cut/pre-processing/GGF_in_SR.h5
data/gluon_jet_1_cut/pre-processing/GGF_in_BR.h5


In [12]:
for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/quark_jet_2_cut/{name}.h5'
    out_h5 = f'./data/quark_jet_2_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/quark_jet_1_cut/{name}.h5'
    out_h5 = f'./data/quark_jet_1_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

data/quark_jet_2_cut/pre-processing/VBF_in_SR.h5
data/quark_jet_2_cut/pre-processing/VBF_in_BR.h5
data/quark_jet_2_cut/pre-processing/GGF_in_SR.h5
data/quark_jet_2_cut/pre-processing/GGF_in_BR.h5
data/quark_jet_1_cut/pre-processing/VBF_in_SR.h5
data/quark_jet_1_cut/pre-processing/VBF_in_BR.h5
data/quark_jet_1_cut/pre-processing/GGF_in_SR.h5
data/quark_jet_1_cut/pre-processing/GGF_in_BR.h5


In [7]:
for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/quark_gluon_jet_2_cut/{name}.h5'
    out_h5 = f'./data/quark_gluon_jet_2_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

data/quark_gluon_jet_2_cut/pre-processing/VBF_in_SR.h5
data/quark_gluon_jet_2_cut/pre-processing/VBF_in_BR.h5
data/quark_gluon_jet_2_cut/pre-processing/GGF_in_SR.h5
data/quark_gluon_jet_2_cut/pre-processing/GGF_in_BR.h5


## $H \to ZZ \to 4\ell$

In [7]:
for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/ZZ_4l/quark_jet_2_cut/{name}.h5'
    out_h5 = f'./data/ZZ_4l/quark_jet_2_cut/pre-processing/{name}.h5'
    to_event_image_h5_ZZ(h5_path, out_h5)

data/ZZ_4l/quark_jet_2_cut/pre-processing/VBF_in_SR.h5
data/ZZ_4l/quark_jet_2_cut/pre-processing/VBF_in_BR.h5
data/ZZ_4l/quark_jet_2_cut/pre-processing/GGF_in_SR.h5
data/ZZ_4l/quark_jet_2_cut/pre-processing/GGF_in_BR.h5


In [5]:
for name in ['VBF', 'GGF']:
    h5_path = f'./data/ZZ_4l/{name}.h5'
    out_h5 = f'./data/ZZ_4l/supervised/pre-processing/{name}.h5'
    to_event_image_h5_ZZ(h5_path, out_h5)

data/ZZ_4l/supervised/pre-processing/VBF.h5
data/ZZ_4l/supervised/pre-processing/GGF.h5


# $p_\text{T}$ smearing

In [3]:
def pt_smearing(h5_path, out_h5):

    out_h5 = Path(out_h5)
    out_h5.parent.mkdir(parents=True, exist_ok=True)

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)
        # Take pT information
        event_pt = np.concatenate([f_out['TOWER/pt'][:], f_out['TRACK/pt'][:], f_out['PHOTON/pt'][:]], axis=1)
            
        # Smearing pt
        event_pt_tmp = np.random.normal(loc=event_pt, scale=pt_scale(event_pt))
        # set negative pt as 0
        event_pt_tmp[event_pt_tmp < 0] = 0
        event_eta = np.concatenate([f_out['TOWER/eta'][:], f_out['TRACK/eta'][:], f_out['PHOTON/eta'][:]], axis=1)
        event_phi = np.concatenate([f_out['TOWER/phi'][:], f_out['TRACK/phi'][:], f_out['PHOTON/phi'][:]], axis=1)

        _, _, new_phi = phi_shift_and_flipping(event_pt_tmp, event_eta, event_phi)

        f_out['TOWER/phi'][:] = new_phi[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/phi'][:] = new_phi[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['PHOTON/phi'][:] = new_phi[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

In [8]:
for n in range(11, 16):
    
    out_dir = Path(f'./data/quark_jet_2_cut/pT_smearing/{n:02}')

    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = f'./data/quark_jet_2_cut/{name}.h5'
        out_h5 = out_dir / f'{name}.h5'
        pt_smearing(h5_path, out_h5)

data/quark_jet_2_cut/pT_smearing/11/VBF_in_SR.h5
data/quark_jet_2_cut/pT_smearing/11/VBF_in_BR.h5
data/quark_jet_2_cut/pT_smearing/11/GGF_in_SR.h5
data/quark_jet_2_cut/pT_smearing/11/GGF_in_BR.h5
data/quark_jet_2_cut/pT_smearing/12/VBF_in_SR.h5
data/quark_jet_2_cut/pT_smearing/12/VBF_in_BR.h5
data/quark_jet_2_cut/pT_smearing/12/GGF_in_SR.h5
data/quark_jet_2_cut/pT_smearing/12/GGF_in_BR.h5
data/quark_jet_2_cut/pT_smearing/13/VBF_in_SR.h5
data/quark_jet_2_cut/pT_smearing/13/VBF_in_BR.h5
data/quark_jet_2_cut/pT_smearing/13/GGF_in_SR.h5
data/quark_jet_2_cut/pT_smearing/13/GGF_in_BR.h5
data/quark_jet_2_cut/pT_smearing/14/VBF_in_SR.h5
data/quark_jet_2_cut/pT_smearing/14/VBF_in_BR.h5
data/quark_jet_2_cut/pT_smearing/14/GGF_in_SR.h5
data/quark_jet_2_cut/pT_smearing/14/GGF_in_BR.h5
data/quark_jet_2_cut/pT_smearing/15/VBF_in_SR.h5
data/quark_jet_2_cut/pT_smearing/15/VBF_in_BR.h5
data/quark_jet_2_cut/pT_smearing/15/GGF_in_SR.h5
data/quark_jet_2_cut/pT_smearing/15/GGF_in_BR.h5


# $\phi$ shifting augmentation

In [3]:
def phi_shifting(h5_path, out_h5, shift_range=np.pi):
    
    out_h5 = Path(out_h5)
    out_h5.parent.mkdir(parents=True, exist_ok=True)

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)

        event_pt = np.concatenate([f_out['TOWER/pt'][:], f_out['TRACK/pt'][:], f_out['PHOTON/pt'][:]], axis=1)
        event_eta = np.concatenate([f_out['TOWER/eta'][:], f_out['TRACK/eta'][:], f_out['PHOTON/eta'][:]], axis=1)
        event_phi = np.concatenate([f_out['TOWER/phi'][:], f_out['TRACK/phi'][:], f_out['PHOTON/phi'][:]], axis=1)

        _, new_eta, new_phi = phi_shift_and_flipping(event_pt, event_eta, event_phi)

        f_out['TOWER/eta'][:] = new_eta[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/eta'][:] = new_eta[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['PHOTON/eta'][:] = new_eta[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

        f_out['TOWER/phi'][:] = new_phi[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/phi'][:] = new_phi[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['PHOTON/phi'][:] = new_phi[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

        # phi shift augmentation
        nevent = f_out['EVENT/type'].shape[0]
        phi_shift = np.random.uniform(-shift_range, shift_range, size=nevent)[:, None]

        f_out['TOWER/phi'][:] += phi_shift
        f_out['TRACK/phi'][:] += phi_shift
        f_out['PHOTON/phi'][:] += phi_shift

        f_out['TOWER/phi'][:] = std_phi(f_out['TOWER/phi'][:])
        f_out['TRACK/phi'][:] = std_phi(f_out['TRACK/phi'][:])
        f_out['PHOTON/phi'][:] = std_phi(f_out['PHOTON/phi'][:])

def phi_shifting_fix(h5_path, out_h5, shift_angle=np.pi):
    
    out_h5 = Path(out_h5)
    out_h5.parent.mkdir(parents=True, exist_ok=True)

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)

        event_pt = np.concatenate([f_out['TOWER/pt'][:], f_out['TRACK/pt'][:], f_out['PHOTON/pt'][:]], axis=1)
        event_eta = np.concatenate([f_out['TOWER/eta'][:], f_out['TRACK/eta'][:], f_out['PHOTON/eta'][:]], axis=1)
        event_phi = np.concatenate([f_out['TOWER/phi'][:], f_out['TRACK/phi'][:], f_out['PHOTON/phi'][:]], axis=1)

        _, new_eta, new_phi = phi_shift_and_flipping(event_pt, event_eta, event_phi)

        f_out['TOWER/eta'][:] = new_eta[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/eta'][:] = new_eta[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['PHOTON/eta'][:] = new_eta[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

        f_out['TOWER/phi'][:] = new_phi[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/phi'][:] = new_phi[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['PHOTON/phi'][:] = new_phi[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

        # phi shift augmentation
        f_out['TOWER/phi'][:] += shift_angle
        f_out['TRACK/phi'][:] += shift_angle
        f_out['PHOTON/phi'][:] += shift_angle

        f_out['TOWER/phi'][:] = std_phi(f_out['TOWER/phi'][:])
        f_out['TRACK/phi'][:] = std_phi(f_out['TRACK/phi'][:])
        f_out['PHOTON/phi'][:] = std_phi(f_out['PHOTON/phi'][:])

In [3]:
for n in range(6, 16):
    
    out_dir = Path(f'./data/quark_jet_2_cut/phi_shifting/{n:02}')

    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = f'./data/quark_jet_2_cut/{name}.h5'
        out_h5 = out_dir / f'{name}.h5'
        phi_shifting(h5_path, out_h5)

data/quark_jet_2_cut/phi_shifting/06/VBF_in_SR.h5
data/quark_jet_2_cut/phi_shifting/06/VBF_in_BR.h5
data/quark_jet_2_cut/phi_shifting/06/GGF_in_SR.h5
data/quark_jet_2_cut/phi_shifting/06/GGF_in_BR.h5
data/quark_jet_2_cut/phi_shifting/07/VBF_in_SR.h5
data/quark_jet_2_cut/phi_shifting/07/VBF_in_BR.h5
data/quark_jet_2_cut/phi_shifting/07/GGF_in_SR.h5
data/quark_jet_2_cut/phi_shifting/07/GGF_in_BR.h5
data/quark_jet_2_cut/phi_shifting/08/VBF_in_SR.h5
data/quark_jet_2_cut/phi_shifting/08/VBF_in_BR.h5
data/quark_jet_2_cut/phi_shifting/08/GGF_in_SR.h5
data/quark_jet_2_cut/phi_shifting/08/GGF_in_BR.h5
data/quark_jet_2_cut/phi_shifting/09/VBF_in_SR.h5
data/quark_jet_2_cut/phi_shifting/09/VBF_in_BR.h5
data/quark_jet_2_cut/phi_shifting/09/GGF_in_SR.h5
data/quark_jet_2_cut/phi_shifting/09/GGF_in_BR.h5
data/quark_jet_2_cut/phi_shifting/10/VBF_in_SR.h5
data/quark_jet_2_cut/phi_shifting/10/VBF_in_BR.h5
data/quark_jet_2_cut/phi_shifting/10/GGF_in_SR.h5
data/quark_jet_2_cut/phi_shifting/10/GGF_in_BR.h5


In [4]:
for angle in range(30, 360, 30):
    
    out_dir = Path(f'./data/quark_jet_2_cut/phi_shifting_fix/{angle}')

    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = f'./data/quark_jet_2_cut/{name}.h5'
        out_h5 = out_dir / f'{name}.h5'
        
        if not out_h5.exists():
            phi_shifting_fix(h5_path, out_h5, angle * np.pi / 180)

data/quark_jet_2_cut/phi_shifting_fix/30/VBF_in_SR.h5
data/quark_jet_2_cut/phi_shifting_fix/30/VBF_in_BR.h5
data/quark_jet_2_cut/phi_shifting_fix/30/GGF_in_SR.h5
data/quark_jet_2_cut/phi_shifting_fix/30/GGF_in_BR.h5
data/quark_jet_2_cut/phi_shifting_fix/60/VBF_in_SR.h5
data/quark_jet_2_cut/phi_shifting_fix/60/VBF_in_BR.h5
data/quark_jet_2_cut/phi_shifting_fix/60/GGF_in_SR.h5
data/quark_jet_2_cut/phi_shifting_fix/60/GGF_in_BR.h5
data/quark_jet_2_cut/phi_shifting_fix/120/VBF_in_SR.h5
data/quark_jet_2_cut/phi_shifting_fix/120/VBF_in_BR.h5
data/quark_jet_2_cut/phi_shifting_fix/120/GGF_in_SR.h5
data/quark_jet_2_cut/phi_shifting_fix/120/GGF_in_BR.h5
data/quark_jet_2_cut/phi_shifting_fix/150/VBF_in_SR.h5
data/quark_jet_2_cut/phi_shifting_fix/150/VBF_in_BR.h5
data/quark_jet_2_cut/phi_shifting_fix/150/GGF_in_SR.h5
data/quark_jet_2_cut/phi_shifting_fix/150/GGF_in_BR.h5
data/quark_jet_2_cut/phi_shifting_fix/210/VBF_in_SR.h5
data/quark_jet_2_cut/phi_shifting_fix/210/VBF_in_BR.h5
data/quark_jet_2_c

# $\eta-\phi$ smearing

In [2]:
def eta_phi_smearing(h5_path, out_h5, std_scale=0.1):
    
    out_h5 = Path(out_h5)
    out_h5.parent.mkdir(parents=True, exist_ok=True)

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)

        event_pt = np.concatenate([f_out['TOWER/pt'][:], f_out['TRACK/pt'][:], f_out['PHOTON/pt'][:]], axis=1)
        event_eta = np.concatenate([f_out['TOWER/eta'][:], f_out['TRACK/eta'][:], f_out['PHOTON/eta'][:]], axis=1)
        event_phi = np.concatenate([f_out['TOWER/phi'][:], f_out['TRACK/phi'][:], f_out['PHOTON/phi'][:]], axis=1)

        std = np.divide(std_scale, event_pt, out=np.zeros_like(event_pt), where=(event_pt!=0))
        event_eta = np.random.normal(loc=event_eta, scale=std)
        event_phi = std_phi(np.random.normal(loc=event_phi, scale=std))

        _, new_eta, new_phi = phi_shift_and_flipping(event_pt, event_eta, event_phi)

        f_out['TOWER/eta'][:] = new_eta[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/eta'][:] = new_eta[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['PHOTON/eta'][:] = new_eta[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

        f_out['TOWER/phi'][:] = new_phi[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/phi'][:] = new_phi[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['PHOTON/phi'][:] = new_phi[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

In [ ]:
for n in range(6, 16):
    
    out_dir = Path(f'./data/quark_jet_2_cut/eta_phi_smearing/{n:02}')

    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = f'./data/quark_jet_2_cut/{name}.h5'
        out_h5 = out_dir / f'{name}.h5'
        eta_phi_smearing(h5_path, out_h5)

data/quark_jet_2_cut/eta_phi_smearing/06/VBF_in_SR.h5
data/quark_jet_2_cut/eta_phi_smearing/06/VBF_in_BR.h5
data/quark_jet_2_cut/eta_phi_smearing/06/GGF_in_SR.h5
data/quark_jet_2_cut/eta_phi_smearing/06/GGF_in_BR.h5
data/quark_jet_2_cut/eta_phi_smearing/07/VBF_in_SR.h5
data/quark_jet_2_cut/eta_phi_smearing/07/VBF_in_BR.h5
data/quark_jet_2_cut/eta_phi_smearing/07/GGF_in_SR.h5
data/quark_jet_2_cut/eta_phi_smearing/07/GGF_in_BR.h5
data/quark_jet_2_cut/eta_phi_smearing/08/VBF_in_SR.h5
data/quark_jet_2_cut/eta_phi_smearing/08/VBF_in_BR.h5
data/quark_jet_2_cut/eta_phi_smearing/08/GGF_in_SR.h5
data/quark_jet_2_cut/eta_phi_smearing/08/GGF_in_BR.h5
data/quark_jet_2_cut/eta_phi_smearing/09/VBF_in_SR.h5
data/quark_jet_2_cut/eta_phi_smearing/09/VBF_in_BR.h5
data/quark_jet_2_cut/eta_phi_smearing/09/GGF_in_SR.h5
data/quark_jet_2_cut/eta_phi_smearing/09/GGF_in_BR.h5
data/quark_jet_2_cut/eta_phi_smearing/10/VBF_in_SR.h5
data/quark_jet_2_cut/eta_phi_smearing/10/VBF_in_BR.h5
data/quark_jet_2_cut/eta_phi

# Pixelation

In [11]:
res = 40
h5_dir = Path('./data/mjj_cut/pre-processing')
npy_dir = Path(f'./data/mjj_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

h5_dir = Path('./data/deta_cut/pre-processing')
npy_dir = Path(f'./data/deta_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

h5_dir = Path('./data/mjj_deta_cut/pre-processing')
npy_dir = Path(f'./data/mjj_deta_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation.py data/mjj_cut/pre-processing/VBF_in_SR.h5 data/mjj_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/mjj_cut/pre-processing/VBF_in_BR.h5 data/mjj_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/mjj_cut/pre-processing/GGF_in_SR.h5 data/mjj_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/mjj_cut/pre-processing/GGF_in_BR.h5 data/mjj_cut/pre-processing/40x40/GGF_in_BR.npy 40 &
python pixelation.py data/deta_cut/pre-processing/VBF_in_SR.h5 data/deta_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/deta_cut/pre-processing/VBF_in_BR.h5 data/deta_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/deta_cut/pre-processing/GGF_in_SR.h5 data/deta_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/deta_cut/pre-processing/GGF_in_BR.h5 data/deta_cut/pre-processing/40x40/GGF_in_BR.npy 40 &
python pixelation.py data/mjj_deta_cut/pre-processing/VBF_in_SR.h5 data/

In [12]:
res = 40
h5_dir = Path('./data/mjj_225_cut/pre-processing')
npy_dir = Path(f'./data/mjj_225_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

h5_dir = Path('./data/deta_2.3_cut/pre-processing')
npy_dir = Path(f'./data/deta_2.3_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

h5_dir = Path('./data/mjj_225_deta_2.3_cut/pre-processing')
npy_dir = Path(f'./data/mjj_225_deta_2.3_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation.py data/mjj_225_cut/pre-processing/VBF_in_SR.h5 data/mjj_225_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/mjj_225_cut/pre-processing/VBF_in_BR.h5 data/mjj_225_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/mjj_225_cut/pre-processing/GGF_in_SR.h5 data/mjj_225_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/mjj_225_cut/pre-processing/GGF_in_BR.h5 data/mjj_225_cut/pre-processing/40x40/GGF_in_BR.npy 40 &
python pixelation.py data/deta_2.3_cut/pre-processing/VBF_in_SR.h5 data/deta_2.3_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/deta_2.3_cut/pre-processing/VBF_in_BR.h5 data/deta_2.3_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/deta_2.3_cut/pre-processing/GGF_in_SR.h5 data/deta_2.3_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/deta_2.3_cut/pre-processing/GGF_in_BR.h5 data/deta_2.3_cut/pre-processing/40x40/GGF_in_BR.npy 40 &
python p

In [4]:
res = 40
h5_dir = Path('./data/supervised/pre-processing')
npy_dir = Path(f'./data/supervised/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF', 'GGF']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation.py data/supervised/pre-processing/VBF.h5 data/supervised/pre-processing/40x40/VBF.npy 40 &
python pixelation.py data/supervised/pre-processing/GGF.h5 data/supervised/pre-processing/40x40/GGF.npy 40 &


## gluon jet

In [9]:
res = 40
h5_dir = Path('./data/gluon_jet_2_cut/pre-processing')
npy_dir = Path(f'./data/gluon_jet_2_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

h5_dir = Path('./data/gluon_jet_1_cut/pre-processing')
npy_dir = Path(f'./data/gluon_jet_1_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation.py data/gluon_jet_2_cut/pre-processing/VBF_in_SR.h5 data/gluon_jet_2_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/gluon_jet_2_cut/pre-processing/VBF_in_BR.h5 data/gluon_jet_2_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/gluon_jet_2_cut/pre-processing/GGF_in_SR.h5 data/gluon_jet_2_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/gluon_jet_2_cut/pre-processing/GGF_in_BR.h5 data/gluon_jet_2_cut/pre-processing/40x40/GGF_in_BR.npy 40 &
python pixelation.py data/gluon_jet_1_cut/pre-processing/VBF_in_SR.h5 data/gluon_jet_1_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/gluon_jet_1_cut/pre-processing/VBF_in_BR.h5 data/gluon_jet_1_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/gluon_jet_1_cut/pre-processing/GGF_in_SR.h5 data/gluon_jet_1_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/gluon_jet_1_cut/pre-processing/GGF_in_BR.h5 data/gluon_je

## quark jet

In [13]:
res = 40
h5_dir = Path('./data/quark_jet_2_cut/pre-processing')
npy_dir = Path(f'./data/quark_jet_2_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

h5_dir = Path('./data/quark_jet_1_cut/pre-processing')
npy_dir = Path(f'./data/quark_jet_1_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation.py data/quark_jet_2_cut/pre-processing/VBF_in_SR.h5 data/quark_jet_2_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/pre-processing/VBF_in_BR.h5 data/quark_jet_2_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/pre-processing/GGF_in_SR.h5 data/quark_jet_2_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/pre-processing/GGF_in_BR.h5 data/quark_jet_2_cut/pre-processing/40x40/GGF_in_BR.npy 40 &
python pixelation.py data/quark_jet_1_cut/pre-processing/VBF_in_SR.h5 data/quark_jet_1_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_1_cut/pre-processing/VBF_in_BR.h5 data/quark_jet_1_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_1_cut/pre-processing/GGF_in_SR.h5 data/quark_jet_1_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/quark_jet_1_cut/pre-processing/GGF_in_BR.h5 data/quark_je

In [8]:
res = 40
h5_dir = Path('./data/quark_gluon_jet_2_cut/pre-processing')
npy_dir = Path(f'./data/quark_gluon_jet_2_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation.py data/quark_gluon_jet_2_cut/pre-processing/VBF_in_SR.h5 data/quark_gluon_jet_2_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/quark_gluon_jet_2_cut/pre-processing/VBF_in_BR.h5 data/quark_gluon_jet_2_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/quark_gluon_jet_2_cut/pre-processing/GGF_in_SR.h5 data/quark_gluon_jet_2_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/quark_gluon_jet_2_cut/pre-processing/GGF_in_BR.h5 data/quark_gluon_jet_2_cut/pre-processing/40x40/GGF_in_BR.npy 40 &


## $p_\text{T}$ smearing

In [9]:
res = 40
for n in range(11, 16):

    h5_dir = Path(f'./data/quark_jet_2_cut/pT_smearing/{n:02}')
    npy_dir = Path(f'./data/quark_jet_2_cut/pT_smearing/{res}x{res}/{n:02}')

    npy_dir.mkdir(parents=True, exist_ok=True)

    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = h5_dir / f'{name}.h5'
        npy_path = npy_dir / f'{name}.npy'
        cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
        print(cmd)

python pixelation.py data/quark_jet_2_cut/pT_smearing/11/VBF_in_SR.h5 data/quark_jet_2_cut/pT_smearing/40x40/11/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/pT_smearing/11/VBF_in_BR.h5 data/quark_jet_2_cut/pT_smearing/40x40/11/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/pT_smearing/11/GGF_in_SR.h5 data/quark_jet_2_cut/pT_smearing/40x40/11/GGF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/pT_smearing/11/GGF_in_BR.h5 data/quark_jet_2_cut/pT_smearing/40x40/11/GGF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/pT_smearing/12/VBF_in_SR.h5 data/quark_jet_2_cut/pT_smearing/40x40/12/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/pT_smearing/12/VBF_in_BR.h5 data/quark_jet_2_cut/pT_smearing/40x40/12/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/pT_smearing/12/GGF_in_SR.h5 data/quark_jet_2_cut/pT_smearing/40x40/12/GGF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/pT_smearing/12/GGF_in_BR.h5 data/quark_je

## $\phi$ shifting

In [4]:
res = 40
for n in range(6, 16):

    h5_dir = Path(f'./data/quark_jet_2_cut/phi_shifting/{n:02}')
    npy_dir = Path(f'./data/quark_jet_2_cut/phi_shifting/{res}x{res}/{n:02}')

    npy_dir.mkdir(parents=True, exist_ok=True)

    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = h5_dir / f'{name}.h5'
        npy_path = npy_dir / f'{name}.npy'
        cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
        print(cmd)

python pixelation.py data/quark_jet_2_cut/phi_shifting/06/VBF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/40x40/06/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting/06/VBF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/40x40/06/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting/06/GGF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/40x40/06/GGF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting/06/GGF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/40x40/06/GGF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting/07/VBF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/40x40/07/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting/07/VBF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/40x40/07/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting/07/GGF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/40x40/07/GGF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting/07/GGF_in_BR.h

## $\eta-\phi$ smearing

In [2]:
res = 40
for n in range(6, 16):

    h5_dir = Path(f'./data/quark_jet_2_cut/eta_phi_smearing/{n:02}')
    npy_dir = Path(f'./data/quark_jet_2_cut/eta_phi_smearing/{res}x{res}/{n:02}')

    npy_dir.mkdir(parents=True, exist_ok=True)


    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = h5_dir / f'{name}.h5'
        npy_path = npy_dir / f'{name}.npy'
        cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
        print(cmd)

python pixelation.py data/quark_jet_2_cut/eta_phi_smearing/06/VBF_in_SR.h5 data/quark_jet_2_cut/eta_phi_smearing/40x40/06/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/eta_phi_smearing/06/VBF_in_BR.h5 data/quark_jet_2_cut/eta_phi_smearing/40x40/06/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/eta_phi_smearing/06/GGF_in_SR.h5 data/quark_jet_2_cut/eta_phi_smearing/40x40/06/GGF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/eta_phi_smearing/06/GGF_in_BR.h5 data/quark_jet_2_cut/eta_phi_smearing/40x40/06/GGF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/eta_phi_smearing/07/VBF_in_SR.h5 data/quark_jet_2_cut/eta_phi_smearing/40x40/07/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/eta_phi_smearing/07/VBF_in_BR.h5 data/quark_jet_2_cut/eta_phi_smearing/40x40/07/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/eta_phi_smearing/07/GGF_in_SR.h5 data/quark_jet_2_cut/eta_phi_smearing/40x40/07/GGF_in_SR.npy 40 &
python pixela

## Without pre-process

In [2]:
res = 40
h5_dir = Path('./data/quark_jet_2_cut')
npy_dir = Path(f'./data/quark_jet_2_cut/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation.py data/quark_jet_2_cut/VBF_in_SR.h5 data/quark_jet_2_cut/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/VBF_in_BR.h5 data/quark_jet_2_cut/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/GGF_in_SR.h5 data/quark_jet_2_cut/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/GGF_in_BR.h5 data/quark_jet_2_cut/40x40/GGF_in_BR.npy 40 &


## Remove photon

In [2]:
res = 40
h5_dir = Path('./data/quark_jet_2_cut/pre-processing')
npy_dir = Path(f'./data/quark_jet_2_cut/pre-processing/remove_photon/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation-remove_photon.py {h5_path} {npy_path} {res} &'
    print(cmd)


python pixelation-remove_photon.py data/quark_jet_2_cut/pre-processing/VBF_in_SR.h5 data/quark_jet_2_cut/pre-processing/remove_photon/40x40/VBF_in_SR.npy 40 &
python pixelation-remove_photon.py data/quark_jet_2_cut/pre-processing/VBF_in_BR.h5 data/quark_jet_2_cut/pre-processing/remove_photon/40x40/VBF_in_BR.npy 40 &
python pixelation-remove_photon.py data/quark_jet_2_cut/pre-processing/GGF_in_SR.h5 data/quark_jet_2_cut/pre-processing/remove_photon/40x40/GGF_in_SR.npy 40 &
python pixelation-remove_photon.py data/quark_jet_2_cut/pre-processing/GGF_in_BR.h5 data/quark_jet_2_cut/pre-processing/remove_photon/40x40/GGF_in_BR.npy 40 &


In [4]:
res = 40
for n in range(1, 16):

    h5_dir = Path(f'./data/quark_jet_2_cut/phi_shifting/{n:02}')
    npy_dir = Path(f'./data/quark_jet_2_cut/phi_shifting/remove_photon/{res}x{res}/{n:02}')

    npy_dir.mkdir(parents=True, exist_ok=True)

    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = h5_dir / f'{name}.h5'
        npy_path = npy_dir / f'{name}.npy'
        cmd = f'python pixelation-remove_photon.py {h5_path} {npy_path} {res} &'
        print(cmd)

python pixelation-remove_photon.py data/quark_jet_2_cut/phi_shifting/01/VBF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon/40x40/01/VBF_in_SR.npy 40 &
python pixelation-remove_photon.py data/quark_jet_2_cut/phi_shifting/01/VBF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon/40x40/01/VBF_in_BR.npy 40 &
python pixelation-remove_photon.py data/quark_jet_2_cut/phi_shifting/01/GGF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon/40x40/01/GGF_in_SR.npy 40 &
python pixelation-remove_photon.py data/quark_jet_2_cut/phi_shifting/01/GGF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon/40x40/01/GGF_in_BR.npy 40 &
python pixelation-remove_photon.py data/quark_jet_2_cut/phi_shifting/02/VBF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon/40x40/02/VBF_in_SR.npy 40 &
python pixelation-remove_photon.py data/quark_jet_2_cut/phi_shifting/02/VBF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon/40x40/02/VBF_in_BR.npy 40 &
python pixelation-remove_photon.py

In [3]:
res = 40
h5_dir = Path('./data/quark_jet_2_cut/pre-processing')
npy_dir = Path(f'./data/quark_jet_2_cut/pre-processing/remove_photon_channel/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation-remove_photon_channel.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation-remove_photon_channel.py data/quark_jet_2_cut/pre-processing/VBF_in_SR.h5 data/quark_jet_2_cut/pre-processing/remove_photon_channel/40x40/VBF_in_SR.npy 40 &
python pixelation-remove_photon_channel.py data/quark_jet_2_cut/pre-processing/VBF_in_BR.h5 data/quark_jet_2_cut/pre-processing/remove_photon_channel/40x40/VBF_in_BR.npy 40 &
python pixelation-remove_photon_channel.py data/quark_jet_2_cut/pre-processing/GGF_in_SR.h5 data/quark_jet_2_cut/pre-processing/remove_photon_channel/40x40/GGF_in_SR.npy 40 &
python pixelation-remove_photon_channel.py data/quark_jet_2_cut/pre-processing/GGF_in_BR.h5 data/quark_jet_2_cut/pre-processing/remove_photon_channel/40x40/GGF_in_BR.npy 40 &


In [5]:
res = 40
for n in range(1, 16):

    h5_dir = Path(f'./data/quark_jet_2_cut/phi_shifting/{n:02}')
    npy_dir = Path(f'./data/quark_jet_2_cut/phi_shifting/remove_photon_channel/{res}x{res}/{n:02}')

    npy_dir.mkdir(parents=True, exist_ok=True)

    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = h5_dir / f'{name}.h5'
        npy_path = npy_dir / f'{name}.npy'
        cmd = f'python pixelation-remove_photon_channel.py {h5_path} {npy_path} {res} &'
        print(cmd)

python pixelation-remove_photon_channel.py data/quark_jet_2_cut/phi_shifting/01/VBF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon_channel/40x40/01/VBF_in_SR.npy 40 &
python pixelation-remove_photon_channel.py data/quark_jet_2_cut/phi_shifting/01/VBF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon_channel/40x40/01/VBF_in_BR.npy 40 &
python pixelation-remove_photon_channel.py data/quark_jet_2_cut/phi_shifting/01/GGF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon_channel/40x40/01/GGF_in_SR.npy 40 &
python pixelation-remove_photon_channel.py data/quark_jet_2_cut/phi_shifting/01/GGF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon_channel/40x40/01/GGF_in_BR.npy 40 &
python pixelation-remove_photon_channel.py data/quark_jet_2_cut/phi_shifting/02/VBF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon_channel/40x40/02/VBF_in_SR.npy 40 &
python pixelation-remove_photon_channel.py data/quark_jet_2_cut/phi_shifting/02/VBF_in_BR.h5 data/quark_jet_2_cut/p

In [2]:
res = 40
case = 3
h5_dir = Path('./data/quark_jet_2_cut/pre-processing')
npy_dir = Path(f'./data/quark_jet_2_cut/pre-processing/remove_photon_case_{case}/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation-remove_photon_info.py {h5_path} {npy_path} {res} {case} &'
    print(cmd)

python pixelation-remove_photon_info.py data/quark_jet_2_cut/pre-processing/VBF_in_SR.h5 data/quark_jet_2_cut/pre-processing/remove_photon_case_3/40x40/VBF_in_SR.npy 40 3 &
python pixelation-remove_photon_info.py data/quark_jet_2_cut/pre-processing/VBF_in_BR.h5 data/quark_jet_2_cut/pre-processing/remove_photon_case_3/40x40/VBF_in_BR.npy 40 3 &
python pixelation-remove_photon_info.py data/quark_jet_2_cut/pre-processing/GGF_in_SR.h5 data/quark_jet_2_cut/pre-processing/remove_photon_case_3/40x40/GGF_in_SR.npy 40 3 &
python pixelation-remove_photon_info.py data/quark_jet_2_cut/pre-processing/GGF_in_BR.h5 data/quark_jet_2_cut/pre-processing/remove_photon_case_3/40x40/GGF_in_BR.npy 40 3 &


In [3]:
res = 40
case = 3
for n in range(1, 16):

    h5_dir = Path(f'./data/quark_jet_2_cut/phi_shifting/{n:02}')
    npy_dir = Path(f'./data/quark_jet_2_cut/phi_shifting/remove_photon_case_{case}/{res}x{res}/{n:02}')

    npy_dir.mkdir(parents=True, exist_ok=True)

    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = h5_dir / f'{name}.h5'
        npy_path = npy_dir / f'{name}.npy'
        cmd = f'python pixelation-remove_photon_info.py {h5_path} {npy_path} {res} {case} &'
        print(cmd)

python pixelation-remove_photon_info.py data/quark_jet_2_cut/phi_shifting/01/VBF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon_case_3/40x40/01/VBF_in_SR.npy 40 3 &
python pixelation-remove_photon_info.py data/quark_jet_2_cut/phi_shifting/01/VBF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon_case_3/40x40/01/VBF_in_BR.npy 40 3 &
python pixelation-remove_photon_info.py data/quark_jet_2_cut/phi_shifting/01/GGF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon_case_3/40x40/01/GGF_in_SR.npy 40 3 &
python pixelation-remove_photon_info.py data/quark_jet_2_cut/phi_shifting/01/GGF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon_case_3/40x40/01/GGF_in_BR.npy 40 3 &
python pixelation-remove_photon_info.py data/quark_jet_2_cut/phi_shifting/02/VBF_in_SR.h5 data/quark_jet_2_cut/phi_shifting/remove_photon_case_3/40x40/02/VBF_in_SR.npy 40 3 &
python pixelation-remove_photon_info.py data/quark_jet_2_cut/phi_shifting/02/VBF_in_BR.h5 data/quark_jet_2_cut/phi_shifting/r

In [3]:
res = 40
case = 1
h5_dir = Path('./data/supervised/pre-processing')
npy_dir = Path(f'./data/supervised/pre-processing/remove_product_case_{case}/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF', 'GGF']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation-remove_decay_product_info.py {h5_path} {npy_path} {res} {case} &'
    print(cmd)

python pixelation-remove_decay_product_info.py data/supervised/pre-processing/VBF.h5 data/supervised/pre-processing/remove_product_case_1/40x40/VBF.npy 40 1 &
python pixelation-remove_decay_product_info.py data/supervised/pre-processing/GGF.h5 data/supervised/pre-processing/remove_product_case_1/40x40/GGF.npy 40 1 &


## $\phi$ shifting with fixed angle

In [5]:
res = 40
for angle in range(30, 360, 30):

    h5_dir = Path(f'./data/quark_jet_2_cut/phi_shifting_fix/{angle}')
    npy_dir = Path(f'./data/quark_jet_2_cut/phi_shifting_fix/{res}x{res}/{angle}')

    npy_dir.mkdir(parents=True, exist_ok=True)

    for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
        h5_path = h5_dir / f'{name}.h5'
        npy_path = npy_dir / f'{name}.npy'
        cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
        print(cmd)

python pixelation.py data/quark_jet_2_cut/phi_shifting_fix/30/VBF_in_SR.h5 data/quark_jet_2_cut/phi_shifting_fix/40x40/30/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting_fix/30/VBF_in_BR.h5 data/quark_jet_2_cut/phi_shifting_fix/40x40/30/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting_fix/30/GGF_in_SR.h5 data/quark_jet_2_cut/phi_shifting_fix/40x40/30/GGF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting_fix/30/GGF_in_BR.h5 data/quark_jet_2_cut/phi_shifting_fix/40x40/30/GGF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting_fix/60/VBF_in_SR.h5 data/quark_jet_2_cut/phi_shifting_fix/40x40/60/VBF_in_SR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting_fix/60/VBF_in_BR.h5 data/quark_jet_2_cut/phi_shifting_fix/40x40/60/VBF_in_BR.npy 40 &
python pixelation.py data/quark_jet_2_cut/phi_shifting_fix/60/GGF_in_SR.h5 data/quark_jet_2_cut/phi_shifting_fix/40x40/60/GGF_in_SR.npy 40 &
python pixela

## $\log(p_\mathrm{T})$

In [2]:
res = 40
h5_dir = Path('./data/quark_jet_2_cut/pre-processing')
npy_dir = Path(f'./data/quark_jet_2_cut/pre-processing/log_pT/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation-log_pT.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation-log_pT.py data/quark_jet_2_cut/pre-processing/VBF_in_SR.h5 data/quark_jet_2_cut/pre-processing/log_pT/40x40/VBF_in_SR.npy 40 &
python pixelation-log_pT.py data/quark_jet_2_cut/pre-processing/VBF_in_BR.h5 data/quark_jet_2_cut/pre-processing/log_pT/40x40/VBF_in_BR.npy 40 &
python pixelation-log_pT.py data/quark_jet_2_cut/pre-processing/GGF_in_SR.h5 data/quark_jet_2_cut/pre-processing/log_pT/40x40/GGF_in_SR.npy 40 &
python pixelation-log_pT.py data/quark_jet_2_cut/pre-processing/GGF_in_BR.h5 data/quark_jet_2_cut/pre-processing/log_pT/40x40/GGF_in_BR.npy 40 &


## $H \to ZZ \to 4\ell$

In [10]:
res = 40
h5_dir = Path('./data/ZZ_4l/quark_jet_2_cut/pre-processing')
npy_dir = Path(f'./data/ZZ_4l/quark_jet_2_cut/pre-processing/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation.py data/ZZ_4l/quark_jet_2_cut/pre-processing/VBF_in_SR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py data/ZZ_4l/quark_jet_2_cut/pre-processing/VBF_in_BR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py data/ZZ_4l/quark_jet_2_cut/pre-processing/GGF_in_SR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py data/ZZ_4l/quark_jet_2_cut/pre-processing/GGF_in_BR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/40x40/GGF_in_BR.npy 40 &


In [11]:
res = 40
case = 4
h5_dir = Path('./data/ZZ_4l/quark_jet_2_cut/pre-processing')
npy_dir = Path(f'./data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_{case}/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation-remove_decay_product_info.py {h5_path} {npy_path} {res} {case} &'
    print(cmd)

python pixelation-remove_decay_product_info.py data/ZZ_4l/quark_jet_2_cut/pre-processing/VBF_in_SR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_4/40x40/VBF_in_SR.npy 40 4 &
python pixelation-remove_decay_product_info.py data/ZZ_4l/quark_jet_2_cut/pre-processing/VBF_in_BR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_4/40x40/VBF_in_BR.npy 40 4 &
python pixelation-remove_decay_product_info.py data/ZZ_4l/quark_jet_2_cut/pre-processing/GGF_in_SR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_4/40x40/GGF_in_SR.npy 40 4 &
python pixelation-remove_decay_product_info.py data/ZZ_4l/quark_jet_2_cut/pre-processing/GGF_in_BR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_4/40x40/GGF_in_BR.npy 40 4 &


In [3]:
res = 40
case = 5
h5_dir = Path('./data/ZZ_4l/quark_jet_2_cut/pre-processing')
npy_dir = Path(f'./data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_{case}/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation-remove_decay_product_info.py {h5_path} {npy_path} {res} {case} &'
    print(cmd)

python pixelation-remove_decay_product_info.py data/ZZ_4l/quark_jet_2_cut/pre-processing/VBF_in_SR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_5/40x40/VBF_in_SR.npy 40 5 &
python pixelation-remove_decay_product_info.py data/ZZ_4l/quark_jet_2_cut/pre-processing/VBF_in_BR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_5/40x40/VBF_in_BR.npy 40 5 &
python pixelation-remove_decay_product_info.py data/ZZ_4l/quark_jet_2_cut/pre-processing/GGF_in_SR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_5/40x40/GGF_in_SR.npy 40 5 &
python pixelation-remove_decay_product_info.py data/ZZ_4l/quark_jet_2_cut/pre-processing/GGF_in_BR.h5 data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_5/40x40/GGF_in_BR.npy 40 5 &


In [7]:
res = 40
case = 4
h5_dir = Path('./data/ZZ_4l/supervised/pre-processing')
npy_dir = Path(f'./data/ZZ_4l/supervised/pre-processing/remove_product_case_{case}/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF', 'GGF']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation-remove_decay_product_info.py {h5_path} {npy_path} {res} {case} &'
    print(cmd)

python pixelation-remove_decay_product_info.py data/ZZ_4l/supervised/pre-processing/VBF.h5 data/ZZ_4l/supervised/pre-processing/remove_product_case_4/40x40/VBF.npy 40 4 &
python pixelation-remove_decay_product_info.py data/ZZ_4l/supervised/pre-processing/GGF.h5 data/ZZ_4l/supervised/pre-processing/remove_product_case_4/40x40/GGF.npy 40 4 &


In [3]:
res = 40
case = 5
h5_dir = Path('./data/ZZ_4l/supervised/pre-processing')
npy_dir = Path(f'./data/ZZ_4l/supervised/pre-processing/remove_product_case_{case}/{res}x{res}')

# create output directory
npy_dir.mkdir(parents=True, exist_ok=True)

for name in ['VBF', 'GGF']:
    h5_path = h5_dir / f'{name}.h5'
    npy_path = npy_dir / f'{name}.npy'
    cmd = f'python pixelation-remove_decay_product_info.py {h5_path} {npy_path} {res} {case} &'
    print(cmd)

python pixelation-remove_decay_product_info.py data/ZZ_4l/supervised/pre-processing/VBF.h5 data/ZZ_4l/supervised/pre-processing/remove_product_case_5/40x40/VBF.npy 40 5 &
python pixelation-remove_decay_product_info.py data/ZZ_4l/supervised/pre-processing/GGF.h5 data/ZZ_4l/supervised/pre-processing/remove_product_case_5/40x40/GGF.npy 40 5 &
